# Enhancer Tools Registry Test Notebook

This notebook is a direct replica of the enhancer tools registry Python script. It demonstrates registration and usage of enhancer tools, ensuring all tool functions accept both string and dictionary input.

In [1]:
# %%
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src


In [2]:
"""
Enhancer Tools Registry: Register and wrap all enhancer-related tools for agent use.
"""
#from __future__ import annotations
from langchain.tools import Tool
from tools.enhancer_agent_tools.keyword_extractor import keyword_extractor_fn
from tools.enhancer_agent_tools.numeric_extractor import extract_numeric_constraints
from tools.enhancer_agent_tools.filter_composer import compose_filters


from typing import List, Dict, Any
from langchain.tools import StructuredTool
from langchain_core.tools import Tool  # ↓ needed only for compose_filters

# ── Pydantic schemas -------------------------------------------------------
from schema.tools_schema import (
    KeywordExtractInput,
    KeywordExtractOutput,
    NumericConstraintInput,
    NumericConstraintOutput,
    FilterComposeInput,
    FilterComposeOutput,
      # alias used inside FilterComposeInput
)


🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src


In [ ]:

def keyword_extractor_tool_func(inputs):
    if isinstance(inputs, dict):
        query = inputs.get("query", "")
    else:
        query = inputs
    return keyword_extractor_fn(query)

def extract_numeric_constraints_tool_func(inputs):
    if isinstance(inputs, dict):
        query = inputs.get("query", "")
    else:
        query = inputs
    return extract_numeric_constraints(query)

def filter_composer_tool_func(inputs):
    if isinstance(inputs, dict):
        tools_outputs = inputs.get("tools_outputs", [])
    else:
        tools_outputs = []
    return compose_filters(*tools_outputs)

keyword_extractor_tool = Tool(
    name="keyword_extractor",
    description="Extracts keywords and maps them to filterable fields from a query string.",
    func=keyword_extractor_tool_func,
)

extract_numeric_constraints_tool = Tool(
    name="extract_numeric_constraints",
    description="Extracts numeric constraints (like funding, revenue, year) from a query string.",
    func=extract_numeric_constraints_tool_func,
)

filter_composer_tool = Tool(
    name="filter_composer",
    description="Composes/merges multiple tool outputs (as JSON strings) into a single filter dictionary.",
    func=filter_composer_tool_func,
)

enhancer_tools = [
    keyword_extractor_tool,
    extract_numeric_constraints_tool,
    filter_composer_tool,
]

In [ ]:
# ────────────────────────────────────────────────────────────
# src/tools/enhancer_registry.py
# Structured versions of:
#   1. keyword_extractor
#   2. extract_numeric_constraints
#   3. filter_composer
# ────────────────────────────────────────────────────────────

# ═══════════════════════════════════════════════════════════════════════════
# 1.  Keyword‑extractor  (string → keywords‑by‑field)
# ═══════════════════════════════════════════════════════════════════════════
def _keyword_extractor_run(payload: KeywordExtractInput) -> KeywordExtractOutput:
    raw = keyword_extractor_fn(payload.text)
    return KeywordExtractOutput(raw)               # wrap in schema

keyword_extractor_tool = StructuredTool.from_function(
    func=_keyword_extractor_run,
    name="keyword_extractor",
    description=(
        "Extracts location / sector / hiring etc. keywords from a user "
        "utterance and maps them to canonical Qdrant payload fields."
    ),
    args_schema=KeywordExtractInput,
)

# ═══════════════════════════════════════════════════════════════════════════
# 2.  Numeric‑constraint extractor  (string → {field: {gte/lte}})
# ═══════════════════════════════════════════════════════════════════════════
def _numeric_constraint_run(    payload: NumericConstraintInput,
) -> NumericConstraintOutput:
    raw = extract_numeric_constraints(payload.text)
    return NumericConstraintOutput(raw)

numeric_constraint_tool = StructuredTool.from_function(
    func=_numeric_constraint_run,
    name="numeric_constraint_extractor",
    description=(
        "Detects phrases like “funding above 50 cr”, “between 2015 and 2020”, "
        "and returns ≤/≥ filters ready for Qdrant."
    ),
    args_schema=NumericConstraintInput,
)

# ═══════════════════════════════════════════════════════════════════════════
# 3.  Filter‑composer  (merge outputs of 1 & 2 or any other tools)
# ═══════════════════════════════════════════════════════════════════════════
def _filter_composer_run(payload: FilterComposeInput) -> FilterComposeOutput:
    # payload.filters is a list of SingleFilterDict                   ↓ convert
    merged: Dict[str, Any] = compose_filters(*[f.root for f in payload.filters])
    return FilterComposeOutput(merged)

filter_composer_tool = StructuredTool.from_function(
    func=_filter_composer_run,
    name="filter_composer",
    description=(
        "Merges multiple filter‑dicts (keyword extractor, numeric extractor, …) "
        "into one final metadata‑filters object."
    ),
    args_schema=FilterComposeInput,
)

# ──────────────────────────────────────────────────────────────
# Convenience list for your Enhancer agent
# ──────────────────────────────────────────────────────────────
enhancer_tools = [
    keyword_extractor_tool,
    numeric_constraint_tool,
    filter_composer_tool,
]


---

**This notebook is a direct copy of the enhancer tools registry Python script, structured for interactive exploration and testing.**